In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical, model_to_dot, plot_model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
import tensorflow_datasets as tfds

In [2]:
dataset = tfds.load("malaria", split = "train")
batch_size = 32
def preprocess_data(example):
    image = example['image']
    image = tf.image.resize(image, [32, 32])
    image = tf.cast(image, tf.float32) / 255.0
    label = example['label']
    return image, label

dataset = dataset.map(preprocess_data)

print(type(dataset))
dataset = dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [3]:
vgg19 = tf.keras.applications.vgg19.VGG19(include_top = False, input_shape = (32, 32, 3), weights = 'imagenet')
vgg19.summary()  ### To show the model's architecture

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [4]:
model_vgg19 = Sequential()
for layer in vgg19.layers:
    layer.trainable = False      ### To turn off VGG19's trainable parameters
model_vgg19.add(vgg19)
model_vgg19.add(Flatten())      
model_vgg19.add(Dense(4096, activation = 'relu'))
model_vgg19.add(Dropout(0.2))
model_vgg19.add(Dense(1024, activation = 'relu'))
model_vgg19.add(Dense(2, activation = 'softmax'))
model_vgg19.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1, 1, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 4096)              2101248   
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              4195328   
                                                                 
 dense_2 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 26,323,010
Trainable params: 6,298,626
Non

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)    
optimizer = tf.keras.optimizers.Adam()

In [6]:
model_vgg19.compile(optimizer=optimizer, loss=loss_object, metrics=['accuracy'])

In [7]:
checkpoint = ModelCheckpoint(
    'malaria.h5',
    monitor='loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=False,
    period=1
)
earlystop = EarlyStopping(
    monitor='loss',
    min_delta=0.001,
    patience=3,
    verbose=1,
    mode='auto'
)
csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)
reduceLR = ReduceLROnPlateau(
    monitor='loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)
callbacks = [checkpoint, earlystop, csvlogger,reduceLR]

In [8]:
### To fit the model
history = model_vgg19.fit(
    dataset, 
    epochs = 10,
    # validation_data = normalized_val_ds, 
    callbacks = callbacks,
    shuffle = True
)


Epoch 1/10


c:\Users\ajbbo\anaconda3\envs\gpu\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


862/862 [==============================] - ETA: 0s - loss: 0.3590 - accuracy: 0.8443
Epoch 1: loss improved from inf to 0.35896, saving model to malaria.h5
862/862 [==============================] - 14s 12ms/step - loss: 0.3590 - accuracy: 0.8443 - lr: 0.0010
Epoch 2/10
859/862 [============================>.] - ETA: 0s - loss: 0.3066 - accuracy: 0.8667
Epoch 2: loss improved from 0.35896 to 0.30649, saving model to malaria.h5
862/862 [==============================] - 10s 11ms/step - loss: 0.3065 - accuracy: 0.8667 - lr: 0.0010
Epoch 3/10
860/862 [============================>.] - ETA: 0s - loss: 0.2925 - accuracy: 0.8740
Epoch 3: loss improved from 0.30649 to 0.29246, saving model to malaria.h5
862/862 [==============================] - 10s 11ms/step - loss: 0.2925 - accuracy: 0.8740 - lr: 0.0010
Epoch 4/10
859/862 [============================>.] - ETA: 0s - loss: 0.2831 - accuracy: 0.8779
Epoch 4: loss improved from 0.29246 to 0.28308, saving model to malaria.h5
862/862 [==========